In [2]:
import xarray as xr
import numpy as np
import geopandas as gpd
import pandas as pd
import re

This notebook (script) is to geolocate the EM-DAT events from 1 to 2023 based on the cleaned GAUL geometries from script 1 (clean_gaul.ipynb(py)).

The first part is locating the events that have the geocodes to the GAUL geometries. 

The second part is locating the events  based on the names of the geometries. For this, the names of the locations and geometries will be matched by a fuzzy algorithm, and then only the geometries that pass a certain threshold and which names exist fully in the location column of the event are kept.
In the case where geometries with similar names at admin levels 1 and 2 are found, the geometries of level 1 are favored and kept over the level 2.

At the end, all geodataframes are concatenated together and columns are created to enable their integration with the GDIS database.


In [3]:
emdat = pd.read_excel('/net/projects/xaida/raw_data/emdat_data/public_emdat_1990_2023.xlsx')


In [4]:
emdat

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update
0,1990-0001-LKA,Yes,nat-hyd-flo-riv,Natural,Hydrological,Flood,Riverine flood,NaN,NaN,LKA,...,NaN,NaN,NaN,NaN,NaN,NaN,42.880732,NaN,2005-12-20,2023-09-25
1,1990-0002-TUN,Yes,nat-hyd-flo-riv,Natural,Hydrological,Flood,Riverine flood,NaN,NaN,TUN,...,NaN,NaN,NaN,NaN,242800.0,566222.0,42.880732,NaN,2006-07-19,2023-09-25
2,1990-0003-WSM,Yes,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,Ofa,WSM,...,NaN,NaN,NaN,NaN,200000.0,466410.0,42.880732,NaN,2005-12-20,2023-09-25
3,1990-0004-FRA,Yes,nat-hyd-mmw-ava,Natural,Hydrological,Mass movement (wet),Avalanche (wet),NaN,NaN,FRA,...,NaN,NaN,NaN,NaN,NaN,NaN,42.880732,NaN,2005-12-21,2023-09-25
4,1990-0005-IDN,Yes,nat-hyd-flo-riv,Natural,Hydrological,Flood,Riverine flood,NaN,NaN,IDN,...,NaN,NaN,NaN,NaN,4800.0,11194.0,42.880732,NaN,2006-07-19,2023-09-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20613,2023-9868-USA,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,USA,...,NaN,NaN,NaN,NaN,14500000.0,14500000.0,100.000000,NaN,2024-01-17,2024-02-27
20614,2023-9872-CHN,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,CHN,...,NaN,NaN,NaN,NaN,2700000.0,2700000.0,100.000000,NaN,2024-02-12,2024-03-04
20615,2023-9873-FSM,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,GLIDE:DR-2023-000130,NaN,FSM,...,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,2024-02-13,2024-04-03
20616,2023-9878-MOZ,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,GLIDE:DR-2024-000073,NaN,MOZ,...,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,2024-06-07,2024-06-14


In [6]:
import pandas as pd

# Read the Excel file into emdat DataFrame and set 'DisNo.' as index
emdat = pd.read_excel('/net/projects/xaida/raw_data/emdat_data/public_emdat_1990_2023.xlsx')
emdat = emdat.set_index('DisNo.')

# Select only 'Location' and 'Admin Units' columns and drop rows where both are NaN
emdat = emdat[['Location', 'Admin Units']].dropna(how='all')

# Sort 'Location' column alphabetically
emdat['Location'] = emdat['Location'].apply(lambda x: ', '.join(sorted(str(x).split(', '))))

# Process 'Admin Units' column to get Admin1 Code, Admin2 Code, and Geo Locations
def process_admin_units(units):
    if pd.isna(units):
        return pd.Series([None, None, None])
    
    admin1_units = []
    admin2_units = []

    # Extract Admin1 and Admin2 information
    for unit in eval(units):
        if 'adm1_code' in unit:
            admin1_units.append((unit['adm1_code'], unit['adm1_name']))
        if 'adm2_code' in unit:
            admin2_units.append((unit['adm2_code'], unit['adm2_name']))
    
    # Sort admin units alphabetically
    admin1_units.sort(key=lambda x: x[1])  # Sort by adm1_name
    admin2_units.sort(key=lambda x: x[1])  # Sort by adm2_name

    admin1_codes = [str(unit[0]) for unit in admin1_units]
    admin1_names_list = [unit[1] for unit in admin1_units]
    
    admin2_codes = [str(unit[0]) for unit in admin2_units]
    admin2_names_list = [unit[1] for unit in admin2_units]
    
    # Construct Geo Locations string
    geo_location_str = ', '.join(admin1_names_list) + (' (Adm1).' if admin1_names_list else '')
    if admin2_names_list:
        geo_location_str += ' ' + ', '.join(admin2_names_list) + ' (Adm2).'
    
    return pd.Series([';'.join(admin1_codes), ';'.join(admin2_codes), geo_location_str])

# Apply the process_admin_units function to 'Admin Units' column and assign the result to Emdata
Emdata = emdat['Admin Units'].apply(process_admin_units)
Emdata.columns = ['Admin1 Code', 'Admin2 Code', 'Geo Locations']

# Merge 'Location' column from emdat into Emdata DataFrame
Emdata['Location'] = emdat['Location']

# Print the DataFrame
print(Emdata)

# Access a specific DisNo.
print(Emdata.loc['2021-0003-IDN'])


              Admin1 Code Admin2 Code Geo Locations  \
DisNo.                                                
1990-0001-LKA        None        None          None   
1990-0002-TUN        None        None          None   
1990-0003-WSM        None        None          None   
1990-0004-FRA        None        None          None   
1990-0005-IDN        None        None          None   
...                   ...         ...           ...   
2023-9651-BRA        None        None          None   
2023-9706-BOL        None        None          None   
2023-9868-USA        None        None          None   
2023-9873-FSM        None        None          None   
2023-9879-BWA        None        None          None   

                                                        Location  
DisNo.                                                            
1990-0001-LKA  Ampara, Badulla, Eliya, Kandy, Kurunegala, Mat...  
1990-0002-TUN  Gabes, Gafsa, Kairouan, Kasserine, Nefta, Sfax...  
1990-0003-WSM   

In [7]:
common_nans = emdat[ emdat['Location'].notna() &  emdat['Admin Units'].isna()]

# Print the result
print("Rows with common NaNs in 'Location' and 'Admin Units':")
print(common_nans)

Rows with common NaNs in 'Location' and 'Admin Units':
                                                        Location Admin Units
DisNo.                                                                      
1990-0001-LKA  Ampara, Badulla, Eliya, Kandy, Kurunegala, Mat...         NaN
1990-0002-TUN  Gabes, Gafsa, Kairouan, Kasserine, Nefta, Sfax...         NaN
1990-0003-WSM                                      Sawali, Upolu         NaN
1990-0004-FRA                                Tignes, Val d'Isere         NaN
1990-0005-IDN  Banyumas municipalities (Central Java), Batang...         NaN
...                                                          ...         ...
2023-9651-BRA                                     Amazonas state         NaN
2023-9706-BOL  Chuquisaca, Cochabamba, La Paz, Oruro, Santa Cruz         NaN
2023-9868-USA  Illinois, Kansas, Louisiana, Missouri, Montana...         NaN
2023-9873-FSM                        Chuuk, Kosrae, Pohnpei, Yap         NaN
2023-9879-BWA        

In [8]:
#get the entires that only have location name indiction (not the geolocation name and code)
Emdata_loconly = Emdata[Emdata['Admin1 Code'].isna() & Emdata['Admin2 Code'].isna()]
#get the geocoded entries
Emdata_geocode = Emdata.drop(Emdata_loconly.index)

## 1) Matching events with geolocations (from 2021 onwards)

### geometries at Admin 1 level

In [9]:
####Administrative level 1 geometries
#Extract Adm1 locations
emdat_geocode_adm1 = Emdata_geocode.loc[Emdata_geocode['Geo Locations'].str.contains("(Adm1)")]
emdat_geocode_adm1["Geo Locations"] = emdat_geocode_adm1["Geo Locations"].str.split(" \(Adm1",expand=True)[0]
#Shape Adm1 locations: 1 row = 1 geolocation
emdat_geocode_adm1_stacked = pd.DataFrame(emdat_geocode_adm1["Admin1 Code"].str.split(';', expand=True).stack()).rename(columns={0:'ADM1_CODE'})
emdat_geocode_adm1_stacked['location'] = pd.DataFrame(emdat_geocode_adm1["Location"].str.split(',', expand=True).stack())
emdat_geocode_adm1_stacked['geolocation'] = pd.DataFrame(emdat_geocode_adm1["Geo Locations"].str.split(',', expand=True).stack())
emdat_geocode_adm1_identified = emdat_geocode_adm1_stacked.reset_index().drop(columns={'level_1'})

emdat_geocode_adm1_identified

/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,DisNo.,ADM1_CODE,location,geolocation
0,1997-0331-EGY,61515,Cairo province,Cairo
1,1999-9388-BDI,40546,Bugabira,Cankuzo
2,1999-9388-BDI,40548,Buganda,Gitega
3,1999-9388-BDI,40549,Busoni,Karuzi
4,1999-9388-BDI,40552,Cankuzo provinces,Makamba
...,...,...,...,...
26204,2023-9167-HND,1438,Jacaleapa,Yoro
26205,2023-9234-ESP,2716,Andalusia,Andalucía
26206,2023-9234-ESP,2724,Asturias,Cataluña/Catalunya
26207,2023-9234-ESP,2718,Catalognia,Principado de Asturias


In [10]:
emdat_geocode_adm1

,Admin1 Code,Admin2 Code,Geo Locations,Location
DisNo.,,,,
1997-0331-EGY,61515,,Cairo,Cairo province
1999-9388-BDI,40546;40548;40549;40552;40554;40557;40558,40638;40605;40639;48356;40642;40643;40610;40596,"Cankuzo, Gitega, Karuzi, Makamba, Muyinga, Rut...","Bugabira, Buganda, Busoni, Cankuzo provinces, ..."
1999-9388-DJI,1093;1094;1095;1096;1097,,"Ali Sabieh, Dikhil, Djibouti, Obock, Tadjourah","Ali Sabieh, Dikhil, Djibouti, Obock, Tadjourah..."
1999-9388-ERI,1205;1208;1211,,"Anseba, Debubawi Keih Bahri, Semenawi Keih Bahri","Anseba, Debubawi Keih Bahri, Semenawi Keih Bah..."
1999-9388-ETH,1230;1232;47678;1236;47679,40799;40800;149281;149298;40803,"Beneshangul Gumu, Gambela, Oromia, SNNPR, Somali","Beneshangul Gumu, Gambela, North Wollo, Oromia..."
...,...,...,...,...
2023-0539-CHL,889;892;149630,12971;12947,"Libertador Gral. Bernardo O'Higgins, Maule, Va...","Biobío Regions, Maule, O’Higgins, Santiago Met..."
2023-0548-IND,1502,,Mizoram,Mizoram State
2023-9167-HND,1420;1421;1433;1438,17413;17439;17335;17287;17313,"Atlantida, Choluteca, Ocotepeque, Yoro","Catacamas, Choculeta, Flores, Jacaleapa, La Es..."


In [11]:
emdat_geocode_adm1['Location'].astype(str).str.split(',').apply(len).sum()

30562

In [12]:
(emdat_geocode_adm1['Admin1 Code'].astype(str).str.split(';').apply(len) + (emdat_geocode_adm1['Admin2 Code'].astype(str).str.split(';').apply(len) - emdat_geocode_adm1['Admin2 Code'].isna())).sum()

34655

In [13]:
emdat_geocode_adm1_identified

,DisNo.,ADM1_CODE,location,geolocation
0,1997-0331-EGY,61515,Cairo province,Cairo
1,1999-9388-BDI,40546,Bugabira,Cankuzo
2,1999-9388-BDI,40548,Buganda,Gitega
3,1999-9388-BDI,40549,Busoni,Karuzi
4,1999-9388-BDI,40552,Cankuzo provinces,Makamba
...,...,...,...,...
26204,2023-9167-HND,1438,Jacaleapa,Yoro
26205,2023-9234-ESP,2716,Andalusia,Andalucía
26206,2023-9234-ESP,2724,Asturias,Cataluña/Catalunya
26207,2023-9234-ESP,2718,Catalognia,Principado de Asturias


In [14]:
gaul_1 = gpd.read_file('/net/projects/xaida/raw_data/gaul_maps/gaul_admin1_clean.gpkg')

In [15]:
gaul_1.columns

Index(['ADM0_CODE', 'ADM0_NAME', 'ADM1_CODE', 'ADM1_NAME', 'DISP_AREA',
       'EXP1_YEAR', 'STATUS', 'STR1_YEAR', 'Shape_Area', 'Shape_Leng',
       'official_name', 'iso3', 'geometry'],
      dtype='object')

In [16]:
gaul_1_loc = gaul_1[['ADM1_CODE','ADM1_NAME']]

In [17]:
gaul_1_loc.dtypes

ADM1_CODE     int64
ADM1_NAME    object
dtype: object

In [18]:
emdat_geocode_adm1_identified.dtypes

DisNo.         object
ADM1_CODE      object
location       object
geolocation    object
dtype: object

In [19]:
emdat_geocode_adm1_identified['ADM1_CODE'] = emdat_geocode_adm1_identified['ADM1_CODE'].astype(int)

In [20]:
emdat_geocode_adm1_identified_merged = emdat_geocode_adm1_identified.merge(gaul_1_loc, on = 'ADM1_CODE',how = 'left')

In [21]:
emdat_geocode_adm1_identified_merged

,DisNo.,ADM1_CODE,location,geolocation,ADM1_NAME
0,1997-0331-EGY,61515,Cairo province,Cairo,Cairo
1,1999-9388-BDI,40546,Bugabira,Cankuzo,Cankuzo
2,1999-9388-BDI,40548,Buganda,Gitega,Gitega
3,1999-9388-BDI,40549,Busoni,Karuzi,Karuzi
4,1999-9388-BDI,40552,Cankuzo provinces,Makamba,Makamba
...,...,...,...,...,...
26214,2023-9167-HND,1438,Jacaleapa,Yoro,Yoro
26215,2023-9234-ESP,2716,Andalusia,Andalucía,Andaluc�a
26216,2023-9234-ESP,2724,Asturias,Cataluña/Catalunya,Catalu�a/Catalunya
26217,2023-9234-ESP,2718,Catalognia,Principado de Asturias,Principado de Asturias


In [22]:
emdat_geocode_adm1_identified_merged["geolocation"] = emdat_geocode_adm1_identified_merged["geolocation"].str.lstrip()
emdat_geocode_adm1_identified_merged["geolocation"] = emdat_geocode_adm1_identified_merged["geolocation"].str.rstrip()

In [23]:
emdat_geocode_adm1_identified_merged["ADM1_NAME"] = emdat_geocode_adm1_identified_merged["ADM1_NAME"].str.lstrip()
emdat_geocode_adm1_identified_merged["ADM1_NAME"] = emdat_geocode_adm1_identified_merged["ADM1_NAME"].str.rstrip()

In [24]:
emdat_geocode_adm1_identified_merged['comparison'] = emdat_geocode_adm1_identified_merged.apply(lambda x: x['geolocation'] == x['ADM1_NAME'], axis=1)

In [25]:
emdat_geocode_adm1_identified_merged[emdat_geocode_adm1_identified_merged.comparison == False]

,DisNo.,ADM1_CODE,location,geolocation,ADM1_NAME,comparison
307,2000-0299-GTM,64834,Jalapa,Quiché,Quich�,False
308,2000-0299-GTM,64828,Quiché,Totonicapán,Totonicap�n,False
322,2000-0335-ISL,1474,Hella city (Rangarvalla province),Rangárvalla,Rang�rvalla,False
353,2000-0359-ESP,2717,Aragón,Aragón,Arag�n,False
354,2000-0359-ESP,2724,Cataluña/Catalunya,Cataluña/Catalunya,Catalu�a/Catalunya,False
...,...,...,...,...,...,...
26089,2023-0280-CAN,835,Québec,Quebec / Québec,Quebec / Qu�bec,False
26145,2023-0365-GTM,64830,Suchitepéques,Suchitepéquez,Suchitep�quez,False
26146,2023-0365-GTM,64828,Zacapa and Totonicapán Departments,Totonicapán,Totonicap�n,False
26215,2023-9234-ESP,2716,Andalusia,Andalucía,Andaluc�a,False


In [120]:
gaul_1[gaul_1.ADM1_CODE == 2754]

,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,DISP_AREA,EXP1_YEAR,STATUS,STR1_YEAR,Shape_Area,Shape_Leng,official_name,iso3,geometry
22,74,South Sudan,2754,Northern Bahr El Ghazal,NO,3000,Member State,1994,2.494846,7.052428,Republic of South Sudan,SSD,"POLYGON ((26.13292 9.08244, 26.13530 9.10099, ..."


In [203]:
gaul_1[gaul_1.ADM0_NAME == 'South Sudan']

,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,DISP_AREA,EXP1_YEAR,STATUS,STR1_YEAR,Shape_Area,Shape_Leng,official_name,iso3,geometry
17,74,South Sudan,2746,El Buheyrat,NO,3000,Member State,1994,3.503620,11.235001,Republic of South Sudan,SSD,"POLYGON ((28.46691 6.47376, 28.46786 6.47947, ..."
18,74,South Sudan,2747,Unity,NO,3000,Member State,1994,3.011566,11.096973,Republic of South Sudan,SSD,"POLYGON ((28.71832 9.04250, 28.71832 9.04375, ..."
19,74,South Sudan,2748,Central Equatoria,NO,3000,Member State,1994,3.595255,11.441561,Republic of South Sudan,SSD,"POLYGON ((29.79232 4.50914, 29.79785 4.51991, ..."
20,74,South Sudan,2750,Eastern Equatoria,NO,3000,Member State,1994,5.769396,13.566082,Republic of South Sudan,SSD,"POLYGON ((31.69433 3.95444, 31.70147 3.96158, ..."
21,74,South Sudan,2751,Jonglei,NO,3000,Member State,1994,9.907790,17.315025,Republic of South Sudan,SSD,"POLYGON ((30.21263 8.91872, 30.21501 8.92348, ..."
22,74,South Sudan,2754,Northern Bahr El Ghazal,NO,3000,Member State,1994,2.494846,7.052428,Republic of South Sudan,SSD,"POLYGON ((26.13292 9.08244, 26.13530 9.10099, ..."
23,74,South Sudan,2765,Warab,NO,3000,Member State,1994,3.105859,9.675830,Republic of South Sudan,SSD,"POLYGON ((27.62626 8.28199, 27.62715 8.28258, ..."
24,74,South Sudan,2766,Western Bahr El Ghazal,NO,3000,Member State,1994,7.362645,15.653986,Republic of South Sudan,SSD,"POLYGON ((24.15070 8.39634, 24.15072 8.39642, ..."
25,74,South Sudan,2768,Western Equatoria,NO,3000,Member State,1994,6.400287,15.338082,Republic of South Sudan,SSD,"POLYGON ((26.28462 6.45896, 26.29016 6.47718, ..."
26,74,South Sudan,37021,Upper Nile,NO,3000,Member State,1994,6.447897,15.496584,Republic of South Sudan,SSD,"POLYGON ((30.74803 9.46688, 30.74923 9.53971, ..."


### geometries at Admin 2 level

In [26]:
####Administrative level 2 geometries
#Extract Adm2 locations
emdat_geocode_adm2 = Emdata_geocode[Emdata_geocode['Geo Locations'].str.contains("(Adm2)")]
#In the Geo Locations column, some rows still have admin1 geometry locations that need to be removed
#clean Adm1 mention for the admin2 locations
emdat_geocode_adm2_cadm1 = emdat_geocode_adm2[emdat_geocode_adm2['Geo Locations'].str.contains("(Adm1)")]#location 2 with location 1 mentions still
#emdat_geocode_adm2_cadm1 = emdat_geocode_adm2_cadm1.reset_index().iloc[:,1::]
emdat_geocode_adm2_cadm1["Geo Locations"] = emdat_geocode_adm2_cadm1["Geo Locations"].str.split(" \(Adm1\).",expand=True)[1].str.replace(" \(Adm2\).","")
emdat_geocode_adm2_wtadm1 = emdat_geocode_adm2[~emdat_geocode_adm2['Geo Locations'].str.contains("(Adm1)")]
emdat_geocode_adm2_wtadm1["Geo Locations"] = emdat_geocode_adm2_wtadm1["Geo Locations"].str.replace(" \(Adm2\).","")

emdat_geocode_adm2 = pd.concat([emdat_geocode_adm2_cadm1,emdat_geocode_adm2_wtadm1])

/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [27]:
emdat_geocode_adm2

,Admin1 Code,Admin2 Code,Geo Locations,Location
DisNo.,,,,
1999-9388-BDI,40546;40548;40549;40552;40554;40557;40558,40638;40605;40639;48356;40642;40643;40610;40596,"Bugabira, Buganda, Busoni, Gihanga, Kirundo, ...","Bugabira, Buganda, Busoni, Cankuzo provinces, ..."
1999-9388-ETH,1230;1232;47678;1236;47679,40799;40800;149281;149298;40803,"North Wollo, Oromia, South Wollo, Southern, W...","Beneshangul Gumu, Gambela, North Wollo, Oromia..."
1999-9388-RWA,21970,21981;21987;21989;21993;21998,"Huye, Muhanga, Ngoma, Nyamagabe, Rubavu","Butare village (Huye district, East/Iburasiraz..."
1999-9388-SOM,2691;2695,25678;25657;25682;25658;25659;25661,"Baardheere, Ceel Barde, Garbahaarey, Rab Dhuu...","Baardheere districts (Gedo province), Bay, Cee..."
2000-0012-MOZ,2114;2115;2120;2121,21850;21853;21855;21883;41375;41376;21885;21886,"Chibuto, Chokwe, Mabalane, Magude, Manhica, M...","Chibuto, Chokwe, Inhambane, Mabalane districts..."
...,...,...,...,...
2023-0391-VEN,,31914,El Callao,El Callao City (Bolivar State)
2023-0451-COL,,13906,Quetame,Quetame Municipality (Cundinamarca Department)
2023-0458-ITA,,18415;18417;18420,"Padova, Treviso, Vicenza","Padoue, Vicence et Trévise regions (Venise)"


In [28]:
emdat_geocode_adm2['Geo Locations'].astype(str).str.split(',').apply(len).sum()

17109

In [29]:
(emdat_geocode_adm2['Admin2 Code'].astype(str).str.split(';')).apply(len).sum()

17109

## 2) Matching events without geolocations, only location names (2021-2023)

In [30]:
#Shape Adm1 locations: 1 row = 1 geolocation
emdat_geocode_adm2_stacked = pd.DataFrame(emdat_geocode_adm2["Admin2 Code"].str.split(';', expand=True).stack()).rename(columns={0:'ADM2_CODE'})
emdat_geocode_adm2_stacked['location'] = pd.DataFrame(emdat_geocode_adm2["Location"].str.split(',', expand=True).stack())
emdat_geocode_adm2_stacked['geolocation'] = pd.DataFrame(emdat_geocode_adm2["Geo Locations"].str.split(',', expand=True).stack())
emdat_geocode_adm2_identified = emdat_geocode_adm2_stacked.reset_index()


##Load administrative boundries objects to merge locations with events

gaul1 = gpd.read_file('/net/projects/xaida/raw_data/gaul_maps/gaul_admin1_clean.gpkg')
gaul2 = gpd.read_file('/net/projects/xaida/raw_data/gaul_maps/gaul_admin2_clean.gpkg')

#get the admin1 geolocated events geometries #emdat_geocode_adm1_identified
emdat_geocode_adm1_identified['ADM1_CODE'] = emdat_geocode_adm1_identified['ADM1_CODE'].astype(int)
events_adm1  = emdat_geocode_adm1_identified.merge(gaul1, on='ADM1_CODE', how='left')

#get the admin2 geolocated events geometries #emdat_geocode_adm2_identified
emdat_geocode_adm2_identified['ADM2_CODE'] = emdat_geocode_adm2_identified['ADM2_CODE'].astype(int)
events_adm2  = emdat_geocode_adm2_identified.merge(gaul2, on='ADM2_CODE', how='left')

#Find the geolocations for the events of which we only have the location name #Emdata_locations_only
#this needs to be done in different steps, as we have no indication about the administrative level of the location
#also, looking up the geometries needs to take place for each country separately, to limit mismatch



In [33]:
#loading data again to work on data without geolocation

Emdata = pd.read_excel('/net/projects/xaida/raw_data/emdat_data/public_emdat_1990_2023.xlsx')
Emdata_loconly = Emdata[Emdata['Admin Units'].isna() & Emdata['Location'].notna()]


In [34]:


def split_and_clean_locations(location):
    entries = re.split(r';', location)
    cleaned_entries = []
    for entry in entries:
        if '(' in entry and ')' in entry:
            cleaned_entries.append(entry.strip())
        else:
            sub_entries = re.split(r',\s*(?![^()]*\))', entry)
            cleaned_entries.extend([sub_entry.strip() for sub_entry in sub_entries if sub_entry.strip()])
    return cleaned_entries

# Expand the DataFrame
expanded_rows = [
    [row['DisNo.'], row['Location'], loc]
    for _, row in Emdata_loconly.iterrows()
    for loc in split_and_clean_locations(row['Location'])
]

expanded_df = pd.DataFrame(expanded_rows, columns=['DisNo.', 'Full_Location_List', 'Individual_Location'])

# Clean the 'Individual_Location' column
replace_terms = ["Near", "Province", "province", "state", "City", "Regency", "provinces", "region", "Region", 
                 "district", "districts", "District", "Municipality", "municipality"]

for term in replace_terms:
    expanded_df['Individual_Location'] = expanded_df['Individual_Location'].str.replace(term, "", regex=False)

def split_text(text):
    if text.count('(') > 1:
        return re.split(r',\s*(?=\S)', re.sub(r'\),\s*(?=\S)', '),\n', text))
    return re.split(r'\),\s*(?=\S)', text)

expanded_df['Individual_Location'] = expanded_df['Individual_Location'].apply(split_text)
expanded_df = expanded_df.explode('Individual_Location', ignore_index=True)

expanded_df['Individual_Location'] = expanded_df['Individual_Location'].str.replace(' and ', ', ')

def extract_locations(row):
    locations = []
    if '(' in row:
        parts = row.split('(')
        locs = parts[0].strip().split(',')
        regions = parts[1].replace(')', '').split(',')
        for loc in locs:
            for sub_loc in loc.strip().split('/'):
                for region in regions:
                    locations.append([sub_loc.strip(), region.strip()])
    elif ',' in row:
        for loc in row.split(','):
            locations.append([loc.strip(), None])
    else:
        for loc in row.strip().split('/'):
            locations.append([loc.strip(), None])
    return locations

new_data = [
    [row['DisNo.'], row['Individual_Location'], loc[0], loc[1]]
    for _, row in expanded_df.iterrows()
    for loc in extract_locations(row['Individual_Location'])
]

new_df = pd.DataFrame(new_data, columns=['DisNo.', 'Individual_Location', 'Location_Before', 'Bracketed'])
new_df['Appended'] = new_df['Location_Before'] + new_df['Bracketed'].apply(lambda x: f" {x}" if x else "")

print(new_df)


              DisNo. Individual_Location Location_Before Bracketed    Appended
0      1990-0001-LKA              Ampara          Ampara      None      Ampara
1      1990-0001-LKA             Badulla         Badulla      None     Badulla
2      1990-0001-LKA               Kandy           Kandy      None       Kandy
3      1990-0001-LKA          Kurunegala      Kurunegala      None  Kurunegala
4      1990-0001-LKA              Matale          Matale      None      Matale
...              ...                 ...             ...       ...         ...
21524  2023-9873-FSM              Kosrae          Kosrae      None      Kosrae
21525  2023-9873-FSM             Pohnpei         Pohnpei      None     Pohnpei
21526  2023-9873-FSM                 Yap             Yap      None         Yap
21527  2023-9879-BWA            Southern        Southern      None    Southern
21528  2023-9879-BWA          North East      North East      None  North East

[21529 rows x 5 columns]


##Now Data with locations , no geocoded data is further divided into three columns Location_Before - Location that is present before (Bracket Admin1 location) and third column appending both location before and Brackted data 

In [35]:
#now data is further divided into two data sets ,first  dataset with  bracket data,
#second dataset is with no bracket data.


new_df_1 = new_df[new_df['Bracketed'].isnull()]
new_df_2 = new_df[~new_df['Bracketed'].isnull()]

In [38]:
new_df_1

,DisNo.,Individual_Location,Location_Before,Bracketed,Appended
0,1990-0001-LKA,Ampara,Ampara,None,Ampara
1,1990-0001-LKA,Badulla,Badulla,None,Badulla
2,1990-0001-LKA,Kandy,Kandy,None,Kandy
3,1990-0001-LKA,Kurunegala,Kurunegala,None,Kurunegala
4,1990-0001-LKA,Matale,Matale,None,Matale
...,...,...,...,...,...
21524,2023-9873-FSM,Kosrae,Kosrae,None,Kosrae
21525,2023-9873-FSM,Pohnpei,Pohnpei,None,Pohnpei
21526,2023-9873-FSM,Yap,Yap,None,Yap
21527,2023-9879-BWA,Southern,Southern,None,Southern


In [39]:
############################
## Functions needed for fuzzy matching of geometries by name
############################

from thefuzz import process
#function for fuzzy names matching
def fuzzy_match(
    df_left, df_right, column_left, column_right, threshold=90, limit=1):
    # Create a series
    series_matches = df_left[column_left].apply(
        lambda x: process.extract(x, df_right[column_right], limit=limit)            # Creates a series with id from df_left and column name _column_left_, with _limit_ matches per item
    )

    # Convert matches to a tidy dataframe
    df_matches = series_matches.to_frame()
    df_matches = df_matches.explode(column_left)     # Convert list of matches to rows
    df_matches[
        ['match_string', 'match_score', 'df_right_id']
    ] = pd.DataFrame(df_matches[column_left].tolist(), index=df_matches.index)       # Convert match tuple to columns
    df_matches.drop(column_left, axis=1, inplace=True)      # Drop column of match tuples

    # Reset index, as in creating a tidy dataframe we've introduced multiple rows per id, so that no longer functions well as the index
    if df_matches.index.name:
        index_name = df_matches.index.name     # Stash index name
    else:
        index_name = 'index'        # Default used by pandas
    df_matches.reset_index(inplace=True)
    df_matches.rename(columns={index_name: 'df_left_id'}, inplace=True)       # The previous index has now become a column: rename for ease of reference

    # Drop matches below threshold
    df_matches.drop(
        df_matches.loc[df_matches['match_score'] < threshold].index,
        inplace=True
    )

    return df_matches

In [40]:
#function for matching datasets uses fuzzy names matching
def match_datasets(df1, df2, col_a, col_b):
    
    df_matches = fuzzy_match(
    df1,
    df2,
    col_a,
    col_b,
    threshold=90,
    limit=1
    )

    df_output = df1.merge(
        df_matches,
        how='left',
        left_index=True,
        right_on='df_left_id'
    ).merge(
        df2,
        how='left',
        left_on='df_right_id',
        right_index=True,
        suffixes=['_df1', '_df2']
    )
    return(df_output)

In [41]:
#function for matching names with corresponding vector geometries
def match_emdat_gdis(emdat_df, gdam_df, iso,varname):
    
    ## EM-DAT preprocess
    emdat_iso = emdat_df[emdat_df["DisNo."].str.contains(iso)]
    emdat_iso = emdat_iso.rename(columns={varname:"name_join"})
    
    gdam_df = gdam_df[gdam_df["iso3"] == iso]
    #join name and alternative names to enhance accuracy
    gdam_df["name_join"] = gdam_df[[varname]].astype(str).agg(' - '.join, axis=1)
    
    if emdat_iso.empty | gdam_df.empty:
        return(gpd.GeoDataFrame())
    else:
        ####double the string to match in EM-DAT for maximal accuracy
        #emdat_iso["name_join"] = emdat_iso[['name_join','name_join']].astype(str).agg(' - '.join, axis=1)
        #Eliminate generic words related to Russian political entities that cause errors in matches
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Rep.","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Kray","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Oblast","")
        #Names of Russian regions that need to be changed
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Dagestan","Daghestan")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Karelya","Karelia")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Yevreyskaya A","Yevrey - Yevreyskaya")
        #Eliminate generic words related to Philippino political entities that cause errors in matches
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region XIII","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region XII","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region XI","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region X","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region IX","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region VIII","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region VII","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region VI","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region VI","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region V","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region III","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region II","")
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("Region I","")   
        emdat_iso["name_join"] = emdat_iso["name_join"].str.replace("region ","")
        
        
        ##GADM preprocess
        
        #Eliminate generic words related to Russian political entities that cause errors in matches
        gdam_df["name_join"] = gdam_df["name_join"].str.replace("Republic of","")
        gdam_df["name_join"] = gdam_df["name_join"].str.replace("Republic","")
        gdam_df["name_join"] = gdam_df["name_join"].str.replace("Respublika","")
        gdam_df["name_join"] = gdam_df["name_join"].str.replace("Kray","")
        gdam_df["name_join"] = gdam_df["name_join"].str.replace("Oblast","")
        
        #Elimination of string in case of no entry
        gdam_df["name_join"] = gdam_df["name_join"].str.replace("- None","")
        
        #Sort both datasets alphabetically with the name join column
        emdat_iso = emdat_iso.sort_values("name_join")
        gdam_df = gdam_df.sort_values("name_join")
        
        #Merge datasets
        merged = match_datasets(emdat_iso, gdam_df, "name_join", "name_join")
        merged = gpd.GeoDataFrame(merged)
        return(merged)

In [42]:
#function for concatenating geopandas dataframes
def concat_db(db1, db2):
    return(pd.concat([db1, db2], axis=0))

In [220]:
#Emdata_locations_only.Location = Emdata_locations_only.Location.str.replace(" Near","").replace(" and ",",").replace(" 'province)","").replace(" 'province ","").replace("Between","").replace('Isl.',"").replace(' districts',"").replace(' region',"")

In [43]:
Emdata_locations_only = new_df_1[['DisNo.','Location_Before']]

Emdata_locations_only.reset_index(drop=True, inplace=True)


iso_locations = Emdata_locations_only['DisNo.'].str.split('-',expand=True)[2]
Emdata_locations_only['iso3'] = iso_locations #add iso3 codes of every location/ event

/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


## 3) Merging all geolocated events

In [44]:
#Administrative level 1
isos = np.unique(Emdata_locations_only["iso3"])#iso codes for the loop
final_db = gpd.GeoDataFrame()#empty geoDF for results

Emdata_locations_only = Emdata_locations_only.rename(columns={'Location_Before':"ADM1_NAME"})#change name of column to merge on

for iso in isos:
    db = match_emdat_gdis(Emdata_locations_only, gaul1, iso,"ADM1_NAME")
    if db.empty:
        pass
    else:
        final_db = pd.concat([final_db,db],axis=0)
        
locations_only_events_admin1 = final_db[~final_db['geometry'].isna()].reset_index().drop(columns={'index'})

/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [45]:
locations_only_events_admin1

,DisNo.,name_join_df1,iso3_df1,df_left_id,match_string,match_score,df_right_id,ADM0_CODE,ADM0_NAME,ADM1_CODE,...,DISP_AREA,EXP1_YEAR,STATUS,STR1_YEAR,Shape_Area,Shape_Leng,official_name,iso3_df2,geometry,name_join_df2
0,1999-0465-AFG,- Badakshan,AFG,7216,Badakhshan,95.0,516.0,1.0,Afghanistan,272.0,...,NO,3000.0,Member State,1000.0,4.398368,20.784230,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",Badakhshan
1,2002-0666-AFG,Badakhshan,AFG,8691,Badakhshan,100.0,516.0,1.0,Afghanistan,272.0,...,NO,3000.0,Member State,1000.0,4.398368,20.784230,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",Badakhshan
2,1998-0267-AFG,Badakhshan,AFG,6008,Badakhshan,100.0,516.0,1.0,Afghanistan,272.0,...,NO,3000.0,Member State,1000.0,4.398368,20.784230,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",Badakhshan
3,1997-0097-AFG,Badakhshan s,AFG,4808,Badakhshan,95.0,516.0,1.0,Afghanistan,272.0,...,NO,3000.0,Member State,1000.0,4.398368,20.784230,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",Badakhshan
4,1993-0169-AFG,Badakhstan,AFG,1950,Badakhshan,90.0,516.0,1.0,Afghanistan,272.0,...,NO,3000.0,Member State,1000.0,4.398368,20.784230,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",Badakhshan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6940,2005-0726-ZWE,Midlands,ZWE,9739,Midlands,100.0,139.0,271.0,Zimbabwe,3444.0,...,NO,3000.0,Member State,1000.0,4.231023,16.425649,Republic of Zimbabwe,ZWE,"POLYGON ((27.99733 -18.23833, 27.99745 -18.238...",Midlands
6941,2022-0569-ZWE,Midlands,ZWE,14429,Midlands,100.0,139.0,271.0,Zimbabwe,3444.0,...,NO,3000.0,Member State,1000.0,4.231023,16.425649,Republic of Zimbabwe,ZWE,"POLYGON ((27.99733 -18.23833, 27.99745 -18.238...",Midlands
6942,2011-0276-ZWE,North,ZWE,11704,Matabeleland North,90.0,34.0,271.0,Zimbabwe,69549.0,...,NO,3000.0,Member State,1997.0,6.474435,16.788383,Republic of Zimbabwe,ZWE,"POLYGON ((25.23703 -17.88990, 25.23757 -17.877...",Matabeleland North
6943,2007-0632-ZWE,South,ZWE,10466,Matabeleland South,90.0,138.0,271.0,Zimbabwe,3443.0,...,NO,3000.0,Member State,1000.0,4.720131,16.330050,Republic of Zimbabwe,ZWE,"POLYGON ((26.69159 -19.89356, 26.69162 -19.893...",Matabeleland South


#consider only geometries where the location string exists in the geometry string name of the gaul object
#events_admin1_locations_only_valid = locations_only_events_admin1[locations_only_events_admin1.apply(lambda x: x['name_join_df2'] in x['name_join_df1'], axis=1)]

In [46]:
events_admin1_locations_only_valid = locations_only_events_admin1[locations_only_events_admin1.match_score >= 90]

In [47]:
events_admin1_locations_only_valid

,DisNo.,name_join_df1,iso3_df1,df_left_id,match_string,match_score,df_right_id,ADM0_CODE,ADM0_NAME,ADM1_CODE,...,DISP_AREA,EXP1_YEAR,STATUS,STR1_YEAR,Shape_Area,Shape_Leng,official_name,iso3_df2,geometry,name_join_df2
0,1999-0465-AFG,- Badakshan,AFG,7216,Badakhshan,95.0,516.0,1.0,Afghanistan,272.0,...,NO,3000.0,Member State,1000.0,4.398368,20.784230,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",Badakhshan
1,2002-0666-AFG,Badakhshan,AFG,8691,Badakhshan,100.0,516.0,1.0,Afghanistan,272.0,...,NO,3000.0,Member State,1000.0,4.398368,20.784230,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",Badakhshan
2,1998-0267-AFG,Badakhshan,AFG,6008,Badakhshan,100.0,516.0,1.0,Afghanistan,272.0,...,NO,3000.0,Member State,1000.0,4.398368,20.784230,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",Badakhshan
3,1997-0097-AFG,Badakhshan s,AFG,4808,Badakhshan,95.0,516.0,1.0,Afghanistan,272.0,...,NO,3000.0,Member State,1000.0,4.398368,20.784230,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",Badakhshan
4,1993-0169-AFG,Badakhstan,AFG,1950,Badakhshan,90.0,516.0,1.0,Afghanistan,272.0,...,NO,3000.0,Member State,1000.0,4.398368,20.784230,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",Badakhshan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6940,2005-0726-ZWE,Midlands,ZWE,9739,Midlands,100.0,139.0,271.0,Zimbabwe,3444.0,...,NO,3000.0,Member State,1000.0,4.231023,16.425649,Republic of Zimbabwe,ZWE,"POLYGON ((27.99733 -18.23833, 27.99745 -18.238...",Midlands
6941,2022-0569-ZWE,Midlands,ZWE,14429,Midlands,100.0,139.0,271.0,Zimbabwe,3444.0,...,NO,3000.0,Member State,1000.0,4.231023,16.425649,Republic of Zimbabwe,ZWE,"POLYGON ((27.99733 -18.23833, 27.99745 -18.238...",Midlands
6942,2011-0276-ZWE,North,ZWE,11704,Matabeleland North,90.0,34.0,271.0,Zimbabwe,69549.0,...,NO,3000.0,Member State,1997.0,6.474435,16.788383,Republic of Zimbabwe,ZWE,"POLYGON ((25.23703 -17.88990, 25.23757 -17.877...",Matabeleland North
6943,2007-0632-ZWE,South,ZWE,10466,Matabeleland South,90.0,138.0,271.0,Zimbabwe,3443.0,...,NO,3000.0,Member State,1000.0,4.720131,16.330050,Republic of Zimbabwe,ZWE,"POLYGON ((26.69159 -19.89356, 26.69162 -19.893...",Matabeleland South


In [48]:
events_admin1_locations_only_valid[['name_join_df1','name_join_df2']].iloc[100:150]

,name_join_df1,name_join_df2
100,Zabul,Zabul
101,Zabul,Zabul
102,Zabul,Zabul
103,Zabul,Zabul
104,Zabul,Zabul
105,Bengo,Bengo
106,Bengo,Bengo
107,Bengo,Bengo
108,Benguela,Benguela
109,Benguela,Benguela


In [228]:
#gaul2['ADM2_NAME_join'] = gaul2[['ADM1_NAME', 'ADM2_NAME']].astype(str).agg('  '.join, axis=1)

In [49]:
gaul2['ADM2_NAME_join'] = gaul2[['ADM2_NAME']]

In [50]:
#Administrative level 2
Emdata_locations_only['ADM2_NAME_join'] = Emdata_locations_only['ADM1_NAME']
final_db2 = gpd.GeoDataFrame()#empty geoDF for results

for iso in isos:
    db2 = match_emdat_gdis(Emdata_locations_only, gaul2, iso,"ADM2_NAME_join")
    if db2.empty:
        pass
    else:
        final_db2 = pd.concat([final_db2,db2],axis=0)
        
locations_only_events_admin2 = final_db2[~final_db2['geometry'].isna()].reset_index().drop(columns={'index'})

/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [51]:
events_admin2_locations_only_valid = locations_only_events_admin2[locations_only_events_admin2.match_score >= 90]

In [53]:
#consider only geometries where the location string exists in the geometry string name of the gaul object
#events_admin2_locations_only_valid = locations_only_events_admin2[locations_only_events_admin2.apply(lambda x: x['name_join_df2'] in x['name_join_df1'], axis=1)]

In [52]:
events_admin2_locations_only_valid

,DisNo.,ADM1_NAME_df1,iso3_df1,name_join_df1,df_left_id,match_string,match_score,df_right_id,ADM0_CODE,ADM0_NAME,...,EXP2_YEAR,STATUS,STR2_YEAR,Shape_Area,Shape_Leng,official_name,iso3_df2,geometry,ADM2_NAME_join,name_join_df2
0,2001-0331-AFG,Aibak),AFG,Aibak),8169,Bak,90.0,16372.0,1.0,Afghanistan,...,3000.0,Member State,1000.0,0.016539,0.726353,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.96592 33.57779, 69.96932 33.58159...",Bak,Bak
1,2003-0375-AFG,Aqcha,AFG,Aqcha,8956,Aqcha,100.0,17494.0,1.0,Afghanistan,...,3000.0,Member State,2004.0,0.015729,0.639598,Islamic Republic of Afghanistan,AFG,"POLYGON ((66.20807 36.96791, 66.20920 36.96242...",Aqcha,Aqcha
2,2005-0326-AFG,Baghlan,AFG,Baghlan,9555,Baghlan-e-Jadid,90.0,16297.0,1.0,Afghanistan,...,3000.0,Member State,2004.0,0.260793,2.642237,Islamic Republic of Afghanistan,AFG,"POLYGON ((68.53223 36.15923, 68.53070 36.16115...",Baghlan-e-Jadid,Baghlan-e-Jadid
3,1997-0097-AFG,Baghlan,AFG,Baghlan,4806,Baghlan-e-Jadid,90.0,16297.0,1.0,Afghanistan,...,3000.0,Member State,2004.0,0.260793,2.642237,Islamic Republic of Afghanistan,AFG,"POLYGON ((68.53223 36.15923, 68.53070 36.16115...",Baghlan-e-Jadid,Baghlan-e-Jadid
4,1998-0267-AFG,Baghlan,AFG,Baghlan,6003,Baghlan-e-Jadid,90.0,16297.0,1.0,Afghanistan,...,3000.0,Member State,2004.0,0.260793,2.642237,Islamic Republic of Afghanistan,AFG,"POLYGON ((68.53223 36.15923, 68.53070 36.16115...",Baghlan-e-Jadid,Baghlan-e-Jadid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6429,2023-0719-ZWE,Wedza,ZWE,Wedza,15382,Hwedza,91.0,2526.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.219286,2.792234,Republic of Zimbabwe,ZWE,"POLYGON ((31.98308 -19.11863, 31.98306 -19.118...",Hwedza,Hwedza
6430,2008-0511-ZWE,Zvimba,ZWE,Zvimba,10832,Zvimba,100.0,2515.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.518949,5.658163,Republic of Zimbabwe,ZWE,"POLYGON ((30.61135 -17.55053, 30.61137 -17.553...",Zvimba,Zvimba
6431,1999-0634-ZWE,Zvimba,ZWE,Zvimba,7429,Zvimba,100.0,2515.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.518949,5.658163,Republic of Zimbabwe,ZWE,"POLYGON ((30.61135 -17.55053, 30.61137 -17.553...",Zvimba,Zvimba
6432,2008-0511-ZWE,Zvishavane,ZWE,Zvishavane,10847,Zvishavane,100.0,10433.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.212085,2.791582,Republic of Zimbabwe,ZWE,"POLYGON ((30.09545 -20.14557, 30.09764 -20.147...",Zvishavane,Zvishavane


In [53]:
events_admin2_locations_only_valid[['name_join_df1','name_join_df2']].iloc[300:350]

,name_join_df1,name_join_df2
300,Abomey-Zou,Abomey
301,Acomey-Calavi,Abomey-calavi
302,Adjohoun,Adjohoun
303,Aguegues villages,Aguegue
304,Allada,Allada
305,Allada,Allada
306,Bante,Bante
307,Bassilla,Bassila
308,Between Ketou,Ketou
309,Between Parakou,Parakou


In [54]:
# Merge the two DataFrames based on the columns 'name_join_df1' and 'DisNo.'
merged_df = pd.merge(events_admin2_locations_only_valid,
                     events_admin1_locations_only_valid,
                     on=['name_join_df1', 'DisNo.'],
                     how='left',
                     suffixes=('', '_right'),
                     indicator=True)

# Keep only the rows from events_admin2_locations_only_valid that are not present in events_admin1_locations_only_valid
events_admin2_locations_only_valid = merged_df.loc[merged_df['_merge'] == 'left_only', events_admin2_locations_only_valid.columns]


In [55]:
events_admin2_locations_only_valid 

,DisNo.,ADM1_NAME_df1,iso3_df1,name_join_df1,df_left_id,match_string,match_score,df_right_id,ADM0_CODE,ADM0_NAME,...,EXP2_YEAR,STATUS,STR2_YEAR,Shape_Area,Shape_Leng,official_name,iso3_df2,geometry,ADM2_NAME_join,name_join_df2
0,2001-0331-AFG,Aibak),AFG,Aibak),8169,Bak,90.0,16372.0,1.0,Afghanistan,...,3000.0,Member State,1000.0,0.016539,0.726353,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.96592 33.57779, 69.96932 33.58159...",Bak,Bak
1,2003-0375-AFG,Aqcha,AFG,Aqcha,8956,Aqcha,100.0,17494.0,1.0,Afghanistan,...,3000.0,Member State,2004.0,0.015729,0.639598,Islamic Republic of Afghanistan,AFG,"POLYGON ((66.20807 36.96791, 66.20920 36.96242...",Aqcha,Aqcha
5,1997-0097-AFG,Balk,AFG,Balk,4804,Balkhab,90.0,17453.0,1.0,Afghanistan,...,3000.0,Member State,1000.0,0.295543,2.820322,Islamic Republic of Afghanistan,AFG,"POLYGON ((66.89229 35.40295, 66.88988 35.40346...",Balkhab,Balkhab
10,1999-0054-AFG,Baraki Barak,AFG,Baraki Barak,6601,Barakibarak,96.0,16386.0,1.0,Afghanistan,...,3000.0,Member State,1000.0,0.026634,0.834392,Islamic Republic of Afghanistan,AFG,"POLYGON ((68.75063 33.91986, 68.75095 33.92102...",Barakibarak,Barakibarak
11,2010-0108-AFG,Col de Salang,AFG,Col de Salang,11264,Salang,90.0,16455.0,1.0,Afghanistan,...,3000.0,Member State,1000.0,0.051562,1.139820,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.24382 35.16703, 69.24326 35.16527...",Salang,Salang
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6453,2023-0719-ZWE,Wedza,ZWE,Wedza,15382,Hwedza,91.0,2526.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.219286,2.792234,Republic of Zimbabwe,ZWE,"POLYGON ((31.98308 -19.11863, 31.98306 -19.118...",Hwedza,Hwedza
6454,2008-0511-ZWE,Zvimba,ZWE,Zvimba,10832,Zvimba,100.0,2515.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.518949,5.658163,Republic of Zimbabwe,ZWE,"POLYGON ((30.61135 -17.55053, 30.61137 -17.553...",Zvimba,Zvimba
6455,1999-0634-ZWE,Zvimba,ZWE,Zvimba,7429,Zvimba,100.0,2515.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.518949,5.658163,Republic of Zimbabwe,ZWE,"POLYGON ((30.61135 -17.55053, 30.61137 -17.553...",Zvimba,Zvimba
6456,2008-0511-ZWE,Zvishavane,ZWE,Zvishavane,10847,Zvishavane,100.0,10433.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.212085,2.791582,Republic of Zimbabwe,ZWE,"POLYGON ((30.09545 -20.14557, 30.09764 -20.147...",Zvishavane,Zvishavane


In [56]:
#for dataset with bracket , repeating the same process , but only in admin 2


Emdata_locations_only = new_df_2[['DisNo.','Appended']]

Emdata_locations_only.reset_index(drop=True, inplace=True)


iso_locations = Emdata_locations_only['DisNo.'].str.split('-',expand=True)[2]
Emdata_locations_only['iso3'] = iso_locations #add iso3 codes of every location/ event

/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [57]:
isos = np.unique(Emdata_locations_only["iso3"])#iso codes for the loop
final_db = gpd.GeoDataFrame()#empty geoDF for results

Emdata_locations_only = Emdata_locations_only.rename(columns={'Appended':"ADM1_NAME"})#change name of column to merge on

for iso in isos:
    db = match_emdat_gdis(Emdata_locations_only, gaul1, iso,"ADM1_NAME")
    if db.empty:
        pass
    else:
        final_db = pd.concat([final_db,db],axis=0)
        
locations_only_events_admin1 = final_db[~final_db['geometry'].isna()].reset_index().drop(columns={'index'})

/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [58]:
events_admin1_locations_only_valid1 = locations_only_events_admin1[locations_only_events_admin1.match_score >= 90]


In [59]:
events_admin1_locations_only_valid1

,DisNo.,name_join_df1,iso3_df1,df_left_id,match_string,match_score,df_right_id,ADM0_CODE,ADM0_NAME,ADM1_CODE,...,DISP_AREA,EXP1_YEAR,STATUS,STR1_YEAR,Shape_Area,Shape_Leng,official_name,iso3_df2,geometry,name_join_df2
0,2012-0361-AFG,Ab Band Ghazni,AFG,4304,Ghazni,90.0,523.0,1.0,Afghanistan,279.0,...,NO,3000.0,Member State,1000.0,2.099038,10.285901,Islamic Republic of Afghanistan,AFG,"POLYGON ((66.82213 33.55940, 66.82243 33.56079...",Ghazni
1,2017-0026-AFG,Afsay Village - Barg-e-Matal Nuristan,AFG,4776,Nuristan,90.0,537.0,1.0,Afghanistan,294.0,...,NO,3000.0,Member State,1000.0,0.892065,6.551736,Islamic Republic of Afghanistan,AFG,"POLYGON ((69.91360 35.35015, 69.91401 35.35259...",Nuristan
2,2018-0398-AFG,Anar Dara Farah,AFG,4952,Farah,90.0,521.0,1.0,Afghanistan,277.0,...,NO,3000.0,Member State,1000.0,4.718270,15.358933,Islamic Republic of Afghanistan,AFG,"POLYGON ((60.58315 33.12982, 60.58746 33.13812...",Farah
3,1991-0130-AFG,Aqcha Jawzjan,AFG,199,Jawzjan,90.0,527.0,1.0,Afghanistan,283.0,...,NO,3000.0,Member State,1000.0,1.121940,6.203613,Islamic Republic of Afghanistan,AFG,"POLYGON ((65.17136 35.94923, 65.17201 35.95074...",Jawzjan
4,1991-0130-AFG,Arabjar Jawzjan,AFG,193,Jawzjan,90.0,527.0,1.0,Afghanistan,283.0,...,NO,3000.0,Member State,1000.0,1.121940,6.203613,Islamic Republic of Afghanistan,AFG,"POLYGON ((65.17136 35.94923, 65.17201 35.95074...",Jawzjan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,2007-0432-ZWE,South Rural s Mashonaland West,ZWE,3595,Mashonaland West,90.0,136.0,271.0,Zimbabwe,3440.0,...,NO,3000.0,Member State,1000.0,4.898301,14.873909,Republic of Zimbabwe,ZWE,"POLYGON ((28.01492 -16.87896, 28.01525 -16.878...",Mashonaland West
2656,2007-0432-ZWE,South Rural s Midlands,ZWE,3596,Midlands,90.0,139.0,271.0,Zimbabwe,3444.0,...,NO,3000.0,Member State,1000.0,4.231023,16.425649,Republic of Zimbabwe,ZWE,"POLYGON ((27.99733 -18.23833, 27.99745 -18.238...",Midlands
2657,2008-0511-ZWE,Zaka Masvingo,ZWE,3763,Masvingo,90.0,137.0,271.0,Zimbabwe,3441.0,...,NO,3000.0,Member State,1000.0,4.898562,12.419403,Republic of Zimbabwe,ZWE,"POLYGON ((29.73057 -21.02383, 29.73119 -21.024...",Masvingo
2658,2009-0645-ZWE,Zaka Masvingo,ZWE,3937,Masvingo,90.0,137.0,271.0,Zimbabwe,3441.0,...,NO,3000.0,Member State,1000.0,4.898562,12.419403,Republic of Zimbabwe,ZWE,"POLYGON ((29.73057 -21.02383, 29.73119 -21.024...",Masvingo


In [60]:
gaul2['ADM2_NAME_join'] = gaul2[['ADM1_NAME', 'ADM2_NAME']].astype(str).agg('  '.join, axis=1)

In [61]:
Emdata_locations_only['ADM2_NAME_join'] = Emdata_locations_only['ADM1_NAME']
final_db2 = gpd.GeoDataFrame()#empty geoDF for results

for iso in isos:
    db2 = match_emdat_gdis(Emdata_locations_only, gaul2, iso,"ADM2_NAME_join")
    if db2.empty:
        pass
    else:
        final_db2 = pd.concat([final_db2,db2],axis=0)
        
locations_only_events_admin2 = final_db2[~final_db2['geometry'].isna()].reset_index().drop(columns={'index'})

/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [62]:
events_admin2_locations_only_valid1 = locations_only_events_admin2[locations_only_events_admin2.match_score >= 90]


In [63]:
events_admin2_locations_only_valid1

,DisNo.,ADM1_NAME_df1,iso3_df1,name_join_df1,df_left_id,match_string,match_score,df_right_id,ADM0_CODE,ADM0_NAME,...,EXP2_YEAR,STATUS,STR2_YEAR,Shape_Area,Shape_Leng,official_name,iso3_df2,geometry,ADM2_NAME_join,name_join_df2
0,2012-0361-AFG,Ab Band Ghazni,AFG,Ab Band Ghazni,4304,Ghazni Ghazni,95.0,16352.0,1.0,Afghanistan,...,3000.0,Member State,1000.0,0.034933,0.988983,Islamic Republic of Afghanistan,AFG,"POLYGON ((68.51923 33.41982, 68.51812 33.41982...",Ghazni Ghazni,Ghazni Ghazni
1,2018-0398-AFG,Anar Dara Farah,AFG,Anar Dara Farah,4952,Farah Farah,95.0,17376.0,1.0,Afghanistan,...,3000.0,Member State,1000.0,0.326498,3.870752,Islamic Republic of Afghanistan,AFG,"POLYGON ((61.88133 32.30833, 61.88319 32.31950...",Farah Farah,Farah Farah
2,1991-0130-AFG,Aqcha Jawzjan,AFG,Aqcha Jawzjan,199,Jawzjan Aqcha,95.0,17494.0,1.0,Afghanistan,...,3000.0,Member State,2004.0,0.015729,0.639598,Islamic Republic of Afghanistan,AFG,"POLYGON ((66.20807 36.96791, 66.20920 36.96242...",Jawzjan Aqcha,Jawzjan Aqcha
3,1993-0169-AFG,Chindawal Kabul,AFG,Chindawal Kabul,632,Kabul Kabul,95.0,16409.0,1.0,Afghanistan,...,3000.0,Member State,1000.0,0.034377,1.168943,Islamic Republic of Afghanistan,AFG,"POLYGON ((68.98100 34.44757, 68.98245 34.44858...",Kabul Kabul,Kabul Kabul
4,2002-0703-AFG,Darwaz Badakshan,AFG,Darwaz Badakshan,2543,Badakhshan Darwaz,92.0,16318.0,1.0,Afghanistan,...,3000.0,Member State,2004.0,0.125913,1.879592,Islamic Republic of Afghanistan,AFG,"POLYGON ((71.02767 37.96882, 71.02387 37.96831...",Badakhshan Darwaz,Badakhshan Darwaz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,2009-0645-ZWE,SEke Mashonaland West,ZWE,SEke Mashonaland West,3926,Mashonaland East Seke,90.0,2531.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.225164,3.924185,Republic of Zimbabwe,ZWE,"POLYGON ((30.76508 -18.50712, 30.76493 -18.504...",Mashonaland East Seke,Mashonaland East Seke
1280,2008-0511-ZWE,Seke Mashonaland East,ZWE,Seke Mashonaland East,3757,Mashonaland East Seke,95.0,2531.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.225164,3.924185,Republic of Zimbabwe,ZWE,"POLYGON ((30.76508 -18.50712, 30.76493 -18.504...",Mashonaland East Seke,Mashonaland East Seke
1281,2008-0511-ZWE,Zaka Masvingo,ZWE,Zaka Masvingo,3763,Masvingo Masvingo,95.0,10421.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.598281,5.458565,Republic of Zimbabwe,ZWE,"POLYGON ((31.40310 -20.00350, 31.39930 -20.009...",Masvingo Masvingo,Masvingo Masvingo
1282,2009-0645-ZWE,Zaka Masvingo,ZWE,Zaka Masvingo,3937,Masvingo Masvingo,95.0,10421.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.598281,5.458565,Republic of Zimbabwe,ZWE,"POLYGON ((31.40310 -20.00350, 31.39930 -20.009...",Masvingo Masvingo,Masvingo Masvingo


In [64]:
events_admin2_locations_only_valid = pd.concat([events_admin2_locations_only_valid1, events_admin2_locations_only_valid])


In [65]:
events_admin2_locations_only_valid

,DisNo.,ADM1_NAME_df1,iso3_df1,name_join_df1,df_left_id,match_string,match_score,df_right_id,ADM0_CODE,ADM0_NAME,...,EXP2_YEAR,STATUS,STR2_YEAR,Shape_Area,Shape_Leng,official_name,iso3_df2,geometry,ADM2_NAME_join,name_join_df2
0,2012-0361-AFG,Ab Band Ghazni,AFG,Ab Band Ghazni,4304,Ghazni Ghazni,95.0,16352.0,1.0,Afghanistan,...,3000.0,Member State,1000.0,0.034933,0.988983,Islamic Republic of Afghanistan,AFG,"POLYGON ((68.51923 33.41982, 68.51812 33.41982...",Ghazni Ghazni,Ghazni Ghazni
1,2018-0398-AFG,Anar Dara Farah,AFG,Anar Dara Farah,4952,Farah Farah,95.0,17376.0,1.0,Afghanistan,...,3000.0,Member State,1000.0,0.326498,3.870752,Islamic Republic of Afghanistan,AFG,"POLYGON ((61.88133 32.30833, 61.88319 32.31950...",Farah Farah,Farah Farah
2,1991-0130-AFG,Aqcha Jawzjan,AFG,Aqcha Jawzjan,199,Jawzjan Aqcha,95.0,17494.0,1.0,Afghanistan,...,3000.0,Member State,2004.0,0.015729,0.639598,Islamic Republic of Afghanistan,AFG,"POLYGON ((66.20807 36.96791, 66.20920 36.96242...",Jawzjan Aqcha,Jawzjan Aqcha
3,1993-0169-AFG,Chindawal Kabul,AFG,Chindawal Kabul,632,Kabul Kabul,95.0,16409.0,1.0,Afghanistan,...,3000.0,Member State,1000.0,0.034377,1.168943,Islamic Republic of Afghanistan,AFG,"POLYGON ((68.98100 34.44757, 68.98245 34.44858...",Kabul Kabul,Kabul Kabul
4,2002-0703-AFG,Darwaz Badakshan,AFG,Darwaz Badakshan,2543,Badakhshan Darwaz,92.0,16318.0,1.0,Afghanistan,...,3000.0,Member State,2004.0,0.125913,1.879592,Islamic Republic of Afghanistan,AFG,"POLYGON ((71.02767 37.96882, 71.02387 37.96831...",Badakhshan Darwaz,Badakhshan Darwaz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6453,2023-0719-ZWE,Wedza,ZWE,Wedza,15382,Hwedza,91.0,2526.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.219286,2.792234,Republic of Zimbabwe,ZWE,"POLYGON ((31.98308 -19.11863, 31.98306 -19.118...",Hwedza,Hwedza
6454,2008-0511-ZWE,Zvimba,ZWE,Zvimba,10832,Zvimba,100.0,2515.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.518949,5.658163,Republic of Zimbabwe,ZWE,"POLYGON ((30.61135 -17.55053, 30.61137 -17.553...",Zvimba,Zvimba
6455,1999-0634-ZWE,Zvimba,ZWE,Zvimba,7429,Zvimba,100.0,2515.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.518949,5.658163,Republic of Zimbabwe,ZWE,"POLYGON ((30.61135 -17.55053, 30.61137 -17.553...",Zvimba,Zvimba
6456,2008-0511-ZWE,Zvishavane,ZWE,Zvishavane,10847,Zvishavane,100.0,10433.0,271.0,Zimbabwe,...,3000.0,Member State,1000.0,0.212085,2.791582,Republic of Zimbabwe,ZWE,"POLYGON ((30.09545 -20.14557, 30.09764 -20.147...",Zvishavane,Zvishavane


### shape geodataframe to match GDIS database

In [66]:
####Shape all geodataframes and set same columns

events_adm1_clean = events_adm1[['DisNo.','iso3','official_name','ADM0_CODE','ADM0_NAME','ADM1_CODE','ADM1_NAME','location', 'geolocation','geometry']]
events_adm1_clean[['ADM2_CODE','ADM2_NAME']] = "None"
events_adm1_clean[['level']] = 1

events_adm2_clean = events_adm2[['DisNo.','iso3','official_name','ADM0_CODE','ADM0_NAME','ADM1_CODE','ADM1_NAME','ADM2_CODE','ADM2_NAME','location', 'geolocation','geometry']]
events_adm2_clean[['level']] = 2

events_adm1_loconly_clean = events_admin1_locations_only_valid.reset_index()[['DisNo.','iso3_df1','official_name','ADM0_CODE','ADM0_NAME','ADM1_CODE','ADM1_NAME','match_string','geometry']]
events_adm1_loconly_clean = events_adm1_loconly_clean.rename(columns={'iso3_df1':'iso3','match_string':'location'})
events_adm1_loconly_clean[['ADM2_CODE','ADM2_NAME','geolocation']] = "None"
events_adm1_loconly_clean[['level']] = 1

events_adm2_loconly_clean = events_admin2_locations_only_valid.reset_index()[['DisNo.','iso3_df1','official_name','ADM0_CODE','ADM0_NAME','ADM1_CODE','ADM1_NAME_df1','ADM2_CODE','ADM2_NAME','match_string','geometry']]
events_adm2_loconly_clean = events_adm2_loconly_clean.rename(columns={'iso3_df1':'iso3','match_string':'location'})
events_adm2_loconly_clean[['geolocation']] = "None"
events_adm2_loconly_clean[['level']] = 2



/net/home/dchinthaparthy/.conda/envs/agg_env/lib/python3.7/site-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


In [67]:
events_adm1_loconly_clean

,DisNo.,iso3,official_name,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,location,geometry,ADM2_CODE,ADM2_NAME,geolocation,level
0,1999-0465-AFG,AFG,Islamic Republic of Afghanistan,1.0,Afghanistan,272.0,Badakhshan,Badakhshan,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",None,None,None,1
1,2002-0666-AFG,AFG,Islamic Republic of Afghanistan,1.0,Afghanistan,272.0,Badakhshan,Badakhshan,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",None,None,None,1
2,1998-0267-AFG,AFG,Islamic Republic of Afghanistan,1.0,Afghanistan,272.0,Badakhshan,Badakhshan,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",None,None,None,1
3,1997-0097-AFG,AFG,Islamic Republic of Afghanistan,1.0,Afghanistan,272.0,Badakhshan,Badakhshan,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",None,None,None,1
4,1993-0169-AFG,AFG,Islamic Republic of Afghanistan,1.0,Afghanistan,272.0,Badakhshan,Badakhshan,"POLYGON ((69.98554 36.78234, 69.98748 36.78352...",None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6940,2005-0726-ZWE,ZWE,Republic of Zimbabwe,271.0,Zimbabwe,3444.0,Midlands,Midlands,"POLYGON ((27.99733 -18.23833, 27.99745 -18.238...",None,None,None,1
6941,2022-0569-ZWE,ZWE,Republic of Zimbabwe,271.0,Zimbabwe,3444.0,Midlands,Midlands,"POLYGON ((27.99733 -18.23833, 27.99745 -18.238...",None,None,None,1
6942,2011-0276-ZWE,ZWE,Republic of Zimbabwe,271.0,Zimbabwe,69549.0,Matabeleland North,Matabeleland North,"POLYGON ((25.23703 -17.88990, 25.23757 -17.877...",None,None,None,1
6943,2007-0632-ZWE,ZWE,Republic of Zimbabwe,271.0,Zimbabwe,3443.0,Matabeleland South,Matabeleland South,"POLYGON ((26.69159 -19.89356, 26.69162 -19.893...",None,None,None,1


In [68]:
events_adm2_loconly_clean

,DisNo.,iso3,official_name,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME_df1,ADM2_CODE,ADM2_NAME,location,geometry,geolocation,level
0,2012-0361-AFG,AFG,Islamic Republic of Afghanistan,1.0,Afghanistan,279.0,Ab Band Ghazni,3525.0,Ghazni,Ghazni Ghazni,"POLYGON ((68.51923 33.41982, 68.51812 33.41982...",None,2
1,2018-0398-AFG,AFG,Islamic Republic of Afghanistan,1.0,Afghanistan,277.0,Anar Dara Farah,3499.0,Farah,Farah Farah,"POLYGON ((61.88133 32.30833, 61.88319 32.31950...",None,2
2,1991-0130-AFG,AFG,Islamic Republic of Afghanistan,1.0,Afghanistan,283.0,Aqcha Jawzjan,99966.0,Aqcha,Jawzjan Aqcha,"POLYGON ((66.20807 36.96791, 66.20920 36.96242...",None,2
3,1993-0169-AFG,AFG,Islamic Republic of Afghanistan,1.0,Afghanistan,284.0,Chindawal Kabul,3585.0,Kabul,Kabul Kabul,"POLYGON ((68.98100 34.44757, 68.98245 34.44858...",None,2
4,2002-0703-AFG,AFG,Islamic Republic of Afghanistan,1.0,Afghanistan,272.0,Darwaz Badakshan,100004.0,Darwaz,Badakhshan Darwaz,"POLYGON ((71.02767 37.96882, 71.02387 37.96831...",None,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,2023-0719-ZWE,ZWE,Republic of Zimbabwe,271.0,Zimbabwe,69550.0,Wedza,33068.0,Hwedza,Hwedza,"POLYGON ((31.98308 -19.11863, 31.98306 -19.118...",None,2
4496,2008-0511-ZWE,ZWE,Republic of Zimbabwe,271.0,Zimbabwe,3440.0,Zvimba,33080.0,Zvimba,Zvimba,"POLYGON ((30.61135 -17.55053, 30.61137 -17.553...",None,2
4497,1999-0634-ZWE,ZWE,Republic of Zimbabwe,271.0,Zimbabwe,3440.0,Zvimba,33080.0,Zvimba,Zvimba,"POLYGON ((30.61135 -17.55053, 30.61137 -17.553...",None,2
4498,2008-0511-ZWE,ZWE,Republic of Zimbabwe,271.0,Zimbabwe,3444.0,Zvishavane,33107.0,Zvishavane,Zvishavane,"POLYGON ((30.09545 -20.14557, 30.09764 -20.147...",None,2


In [69]:
new_df

,DisNo.,Individual_Location,Location_Before,Bracketed,Appended
0,1990-0001-LKA,Ampara,Ampara,None,Ampara
1,1990-0001-LKA,Badulla,Badulla,None,Badulla
2,1990-0001-LKA,Kandy,Kandy,None,Kandy
3,1990-0001-LKA,Kurunegala,Kurunegala,None,Kurunegala
4,1990-0001-LKA,Matale,Matale,None,Matale
...,...,...,...,...,...
21524,2023-9873-FSM,Kosrae,Kosrae,None,Kosrae
21525,2023-9873-FSM,Pohnpei,Pohnpei,None,Pohnpei
21526,2023-9873-FSM,Yap,Yap,None,Yap
21527,2023-9879-BWA,Southern,Southern,None,Southern


In [70]:
freq_emdata = new_df['DisNo.'].value_counts()

# Count the frequency of each DisNo. in locations_only_events_admin1
freq_admin1 = events_adm1_loconly_clean['DisNo.'].value_counts()

# Count the frequency of each DisNo. in vents_adm2_loconly_clean
freq_admin2 = events_adm2_loconly_clean['DisNo.'].value_counts()

# Combine the frequencies into a single DataFrame
combined_freq = pd.DataFrame({
    'Emdata': freq_emdata, 
    'Admin1': freq_admin1, 
    'Admin2': freq_admin2
}).fillna(0)

# Calculate the ratio for each DisNo.
combined_freq['Ratio_Admin1'] = combined_freq['Admin1'] / combined_freq['Emdata']
combined_freq['Ratio_Admin2'] = combined_freq['Admin2'] / combined_freq['Emdata']
combined_freq['total_ratio'] =  combined_freq['Ratio_Admin1'] + combined_freq['Ratio_Admin2'] 
# Display the result
print(combined_freq)


               Emdata  Admin1  Admin2  Ratio_Admin1  Ratio_Admin2  total_ratio
1990-0001-LKA      10     0.0     9.0          0.00           0.9         0.90
1990-0002-TUN       8     6.0     0.0          0.75           0.0         0.75
1990-0003-WSM       2     0.0     0.0          0.00           0.0         0.00
1990-0004-FRA       2     0.0     1.0          0.00           0.5         0.50
1990-0005-IDN      11     0.0     0.0          0.00           0.0         0.00
...               ...     ...     ...           ...           ...          ...
2023-9651-BRA       1     1.0     0.0          1.00           0.0         1.00
2023-9706-BOL       5     5.0     0.0          1.00           0.0         1.00
2023-9868-USA      10    10.0     0.0          1.00           0.0         1.00
2023-9873-FSM       4     0.0     0.0          0.00           0.0         0.00
2023-9879-BWA       2     2.0     0.0          1.00           0.0         1.00

[10887 rows x 6 columns]


In [71]:
disno_to_remove = combined_freq[combined_freq['total_ratio'] < 0.6].index

# Filter out these DisNo. from events_adm1_loconly_clean and events_adm2_loconly_clean
events_adm1_loconly_clean = events_adm1_loconly_clean[~events_adm1_loconly_clean['DisNo.'].isin(disno_to_remove)].reset_index(drop=True)
events_adm2_loconly_clean = events_adm2_loconly_clean[~events_adm2_loconly_clean['DisNo.'].isin(disno_to_remove)].reset_index(drop=True)

# Display the result
print("Filtered events_adm1_loconly_clean:")
print(events_adm1_loconly_clean)
print("\nFiltered events_adm2_loconly_clean:")
print(events_adm2_loconly_clean)

Filtered events_adm1_loconly_clean:
             DisNo. iso3                    official_name  ADM0_CODE  \
0     1999-0465-AFG  AFG  Islamic Republic of Afghanistan        1.0   
1     2002-0666-AFG  AFG  Islamic Republic of Afghanistan        1.0   
2     1998-0267-AFG  AFG  Islamic Republic of Afghanistan        1.0   
3     1997-0097-AFG  AFG  Islamic Republic of Afghanistan        1.0   
4     1993-0169-AFG  AFG  Islamic Republic of Afghanistan        1.0   
...             ...  ...                              ...        ...   
6408  2023-0719-ZWE  ZWE             Republic of Zimbabwe      271.0   
6409  2005-0726-ZWE  ZWE             Republic of Zimbabwe      271.0   
6410  2011-0276-ZWE  ZWE             Republic of Zimbabwe      271.0   
6411  2007-0632-ZWE  ZWE             Republic of Zimbabwe      271.0   
6412  2005-0726-ZWE  ZWE             Republic of Zimbabwe      271.0   

        ADM0_NAME  ADM1_CODE           ADM1_NAME            location  \
0     Afghanistan      272.

In [72]:
geolocated_events = pd.concat([events_adm1_clean, events_adm2_clean,events_adm1_loconly_clean,events_adm2_loconly_clean])
geolocated_events['disaster_number_country'] = geolocated_events['DisNo.']
geolocated_events = geolocated_events.set_index('DisNo.').sort_index()

In [73]:
geolocated_events[['ADM0_CODE','ADM1_CODE']] = geolocated_events[['ADM0_CODE','ADM1_CODE']].astype(int)

In [74]:
gaul1_subset = gaul1[['ADM1_CODE','ADM1_NAME']]

In [75]:
def find_location(x):
    return gaul1_subset[gaul1_subset.ADM1_CODE == x]['ADM1_NAME'].values[0]

In [76]:
ADM1_NAMES_data = geolocated_events['ADM1_CODE'].apply(lambda x: find_location(x))

In [77]:
ADM1_NAMES_data

DisNo.
1990-0001-LKA    North Central
1990-0001-LKA          Central
1990-0001-LKA          Central
1990-0001-LKA    North Western
1990-0001-LKA          Central
                     ...      
2023-9868-USA            Texas
2023-9868-USA       Washington
2023-9868-USA         Missouri
2023-9879-BWA         Southern
2023-9879-BWA       North East
Name: ADM1_CODE, Length: 53520, dtype: object

In [78]:
geolocated_events['ADM1_NAME'] = ADM1_NAMES_data

In [79]:
geolocated_events = geolocated_events.drop(columns='ADM1_NAME_df1')

In [80]:
len(np.unique(geolocated_events.index))

13557

In [81]:
print(geolocated_events)

              iso3                               official_name  ADM0_CODE  \
DisNo.                                                                      
1990-0001-LKA  LKA  Democratic Socialist Republic of Sri Lanka        231   
1990-0001-LKA  LKA  Democratic Socialist Republic of Sri Lanka        231   
1990-0001-LKA  LKA  Democratic Socialist Republic of Sri Lanka        231   
1990-0001-LKA  LKA  Democratic Socialist Republic of Sri Lanka        231   
1990-0001-LKA  LKA  Democratic Socialist Republic of Sri Lanka        231   
...            ...                                         ...        ...   
2023-9868-USA  USA                    United States of America        259   
2023-9868-USA  USA                    United States of America        259   
2023-9868-USA  USA                    United States of America        259   
2023-9879-BWA  BWA                        Republic of Botswana         35   
2023-9879-BWA  BWA                        Republic of Botswana         35   

In [82]:
geolocated_events.columns

Index(['iso3', 'official_name', 'ADM0_CODE', 'ADM0_NAME', 'ADM1_CODE',
       'ADM1_NAME', 'location', 'geolocation', 'geometry', 'ADM2_CODE',
       'ADM2_NAME', 'level', 'disaster_number_country'],
      dtype='object')

In [83]:
geolocated_events = gpd.GeoDataFrame(geolocated_events)
geolocated_events.to_file('geolocated_events_clean_1990-2023_final.gpkg', driver="GPKG")

In [84]:
!ls -lh



total 12G
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy  690K May 23 15:41 02_2023.ipynb
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy  573K May 18 16:12 02_geolocate_events-2.ipynb
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy  593K Apr 24 23:12 02_geolocate_events-Copy1.ipynb
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy  719K Jul 11 12:28 02_geolocate_events-mod.ipynb
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy  736K May 29 00:30 02_geolocate_events-no-locations.ipynb
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy  678K May 19 15:05 02_geolocate_events.ipynb
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy  874K Jul 11 13:06 02_geolocate_events_full.ipynb
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy  148K May 22 16:31 2023.xlsx
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy   442 Feb 28 17:14 README.md
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy   983 Feb 29 15:28 Untitled.ipynb
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy  2.0K Apr 24 22:54 Untitled1.ipynb
-rw-rw-r-- 1 dchinthaparthy dchinthaparthy  298K May 29 2

In [85]:
cp geolocated_events_clean_1990-2023_final.gpkg /net/scratch/XAIDA/share_folder


In [89]:
data = gpd.read_file('geolocated_events_clean_1990-2023_final.gpkg')

In [90]:
data

,DisNo.,iso3,official_name,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,location,geolocation,ADM2_CODE,ADM2_NAME,level,disaster_number_country,geometry
0,1990-0001-LKA,LKA,Democratic Socialist Republic of Sri Lanka,231,Sri Lanka,2738,North Central,Polonnaruwa,None,25836,Polonnaruwa,2,1990-0001-LKA,"POLYGON ((80.98834 7.72038, 80.98802 7.72037, ..."
1,1990-0001-LKA,LKA,Democratic Socialist Republic of Sri Lanka,231,Sri Lanka,2736,Central,Nuwara Eliya,None,41749,Nuwara Eliya,2,1990-0001-LKA,"POLYGON ((80.46715 7.02203, 80.46714 7.02214, ..."
2,1990-0001-LKA,LKA,Democratic Socialist Republic of Sri Lanka,231,Sri Lanka,2736,Central,Kandy,None,41748,Kandy,2,1990-0001-LKA,"POLYGON ((80.56631 7.45748, 80.56660 7.45725, ..."
3,1990-0001-LKA,LKA,Democratic Socialist Republic of Sri Lanka,231,Sri Lanka,2739,North Western,Kurunegala,None,25837,Kurunegala,2,1990-0001-LKA,"POLYGON ((80.49657 7.80912, 80.49659 7.80868, ..."
4,1990-0001-LKA,LKA,Democratic Socialist Republic of Sri Lanka,231,Sri Lanka,2736,Central,Matale,None,25830,Matale,2,1990-0001-LKA,"POLYGON ((80.49657 7.80912, 80.49657 7.80952, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53515,2023-9868-USA,USA,United States of America,259,United States of America,3257,Texas,Texas,None,None,None,1,2023-9868-USA,"MULTIPOLYGON (((-97.51395 26.83975, -97.51390 ..."
53516,2023-9868-USA,USA,United States of America,259,United States of America,3261,Washington,Washington,None,None,None,1,2023-9868-USA,"MULTIPOLYGON (((-124.72645 47.99174, -124.7265..."
53517,2023-9868-USA,USA,United States of America,259,United States of America,3239,Missouri,Missouri,None,None,None,1,2023-9868-USA,"POLYGON ((-95.77470 40.57357, -95.77355 40.578..."
53518,2023-9879-BWA,BWA,Republic of Botswana,35,Botswana,663,Southern,Southern,None,None,None,1,2023-9879-BWA,"POLYGON ((23.04889 -24.83722, 23.04917 -24.823..."
